In [1]:
import os
os.chdir('..')

In [2]:
from dataclasses import  dataclass
from pathlib import  Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path

In [3]:
from mlproject.constants import  *
from mlproject.utils.common import  read_yaml, create_directories

In [4]:
class ConfigurationManager:
    def __init__(self,config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH,schema_filepath=SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self):
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=Path(config.root_dir),
            data_path=Path(config.data_path)
        )
        return  data_transformation_config

In [5]:
import os
from mlproject import logger
from sklearn.model_selection import train_test_split
import  pandas as pd

In [6]:
class DataTransformation:
    def __init__(self,config:DataTransformationConfig):
        self.config = config

    def train_test_split(self):
        data = pd.read_csv(self.config.data_path)

        train, test = train_test_split(data)

        train.to_csv(os.path.join(self.config.root_dir,"train.csv"),index=False)
        test.to_csv(os.path.join(self.config.root_dir, "test.csv"),index=False)

        logger.info("Split data into training and test sets")
        logger.info(f"Train set shape: {train.shape}")
        logger.info(f"Test set shape: {test.shape}")


In [7]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.train_test_split()
except Exception as e:
    raise e


[2026-02-20 07:47:42,481]: INFO: common: Yaml file: config\config.yaml loaded successfully
[2026-02-20 07:47:42,484]: INFO: common: Yaml file: params.yaml loaded successfully
[2026-02-20 07:47:42,486]: INFO: common: Yaml file: schema.yaml loaded successfully
[2026-02-20 07:47:42,489]: INFO: common: Created directory at: artifacts
[2026-02-20 07:47:42,490]: INFO: common: Created directory at: artifacts/data_transformation
[2026-02-20 07:47:42,551]: INFO: 3768698132: Split data into training and test sets
[2026-02-20 07:47:42,551]: INFO: 3768698132: Train set shape: (1199, 12)
[2026-02-20 07:47:42,552]: INFO: 3768698132: Test set shape: (400, 12)
